In [1]:
from sklearn.preprocessing import normalize
from sklearn.feature_selection import chi2,mutual_info_regression
import scipy.cluster.hierarchy as shc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score

In [1]:
import random
import pandas as pd
import numpy as np

def cal_Cmass(data):
    '''
    input:data(ndarray)
    output:mass(ndarray)
    '''
    Cmass = np.mean(data,axis=0)
    return Cmass

def distance(x, y, p=2):
    '''
    input:x(ndarray)
          y(ndarray)
          p(int)
    output:distance(float)   
    '''   
    dis2 = np.sum(np.abs(x-y)**p)
    dis = np.power(dis2,1/p)
    return dis


def total_inertia(data):
    '''
    input:data(ndarray)
          Cmass(ndarray)
    output:dis_list(list)
    '''
    dis_list = []
    Cmass = cal_Cmass(data)
    for i in range(len(data)): 
        dis_list.append(distance(Cmass,data[i][:]))
    dis_list = sum(dis_list)
    return dis_list

data=pd.read_csv("brain_perplexity40_tsne2d_hier-clust_maxclust127.csv")
inertia = []
for i in np.unique(data['tsne2d_hier_clust']):
    tmp = data.loc[data['tsne2d_hier_clust']==i]
    inertia.append([total_inertia(tmp.iloc[:,20:22].values)/(np.power(tmp.shape[0],2)),tmp.shape[0]])
inertia_res=pd.DataFrame(inertia,index=np.unique(data['tsne2d_hier_clust']),columns=['total_inertia','gene_numbers'])

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import copy
from sklearn import datasets
 
 
def find_neighbor(j, x, eps):
    """
    :param j: index of the core point
    :param x: input data
    :param eps: radius of neighbourhood
    """
    temp = np.sum((x - x[j]) ** 2, axis=1) ** 0.5
    N = np.argwhere(temp <= eps).flatten().tolist()
    return N
 
 
def seed_search(X, omega_list, eps, MinPts):
    k = -1
    neighbor_list = []

    gama = set([x for x in range(len(X))])
    cluster = [-1 for _ in range(len(X))]
 
    for i in range(len(X)):
        neighbor_list.append(find_neighbor(i, X, eps))
 
    omega_list = set(omega_list)
    while len(omega_list) > 0:
        gama_old = copy.deepcopy(gama)
        j = random.choice(list(omega_list))
        k = k + 1
        Q = list()
        Q.append(j)
        gama.remove(j)
        while len(Q) > 0:
            q = Q[0]
            Q.remove(q)
            if len(neighbor_list[q]) >= MinPts:
                delta = set(neighbor_list[q]) & gama
                delta_list = list(delta)
                for i in range(len(delta)):
                    Q.append(delta_list[i])
                    gama = gama - delta
        Ck = gama_old - gama
        Cklist = list(Ck)
        for i in range(len(Ck)):
            cluster[Cklist[i]] = k
        omega_list = omega_list - Ck
    return cluster

In [2]:
magma=pd.read_csv("brain_perplexity40_tsne2d_hier-clust_maxclust127_magmagene.csv",sep=",")
omega_list=magma.loc[magma['magma_gene']]['Symbol'].index
X1=magma.iloc[:,19:21].values

In [3]:
omega_list

Int64Index([   36,   137,   235,   268,   807,   808,   810,   812,   895,
             1245,  1276,  1340,  1356,  1371,  1423,  1441,  1706,  2045,
             2049,  2074,  2352,  2541,  2542,  2868,  2876,  2972,  2986,
             3032,  3276,  3277,  4593,  4640,  4751,  4848,  5096,  5485,
             5620,  5892,  7216,  7460,  8406,  8700,  8978,  8979,  8980,
             8982,  8983,  9425,  9443, 10783, 10810, 11278, 11308, 11558,
            11561, 11771, 11772, 12149, 12820, 13381, 13662, 13869, 14190,
            14539, 15122, 15316, 15424, 15430, 15908, 16048, 16080, 16160,
            16606, 16650, 16740, 16777, 17002, 17197],
           dtype='int64')

In [4]:
eps = 0.5
min_Pts = 6
C = seed_search(X1, omega_list, eps, min_Pts)
magma['gwasseed_clust']=C

In [5]:
len([x for x in C if x != -1])

873

In [6]:
len(np.unique(C))

79

In [7]:
magma.to_csv("brain_embedding_clusters_maxclust127_gwasseed-clust-eps0.5pts6.csv")